To run this Fenic demo, click **Runtime** > **Run all**.

<div class="align-center">
<a href="https://github.com/typedef-ai/fenic"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/typedef-fenic-logo-github-yellow.png" height="50"></a>
<a href="https://discord.gg/GdqF3J7huR"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/join-the-discord.png" height="50"></a>
<a href="https://docs.fenic.ai/latest/"><img src="https://github.com/typedef-ai/fenic/blob/main/docs/images/documentation.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/typedef-ai/fenic).

</div>

In [ ]:
!pip uninstall -y sklearn-compat ibis-framework imbalanced-learn google-genai
!pip install polars==1.30.0
# === GOOGLE GEMINI ===
#!pip install fenic[google]
# === ANTHROPIC CLAUDE ===
#!pip install fenic[anthropic]
# === OPENAI (Default) ===
!pip install fenic

In [ ]:
import os 
import getpass

# 🔌 MULTI-PROVIDER SETUP - Choose your preferred LLM provider
# Uncomment ONE of the provider sections below:

# === OPENAI (Default) ===
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# === GOOGLE GEMINI ===
# os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google API Key:")

# === ANTHROPIC CLAUDE ===
# os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API Key:")

# 🛡️ Content Moderation

**Hook:** *"AI content moderation that understands context, not just keywords"*

Traditional content moderation relies on keyword filters that miss sophisticated bad actors. "Make money from home" could be legitimate advice or a romance scam setup. AI content moderation understands context, intent, and subtle manipulation tactics that rule-based systems completely miss.

**What you'll see in this 2-minute demo:**
- 📱 **Dating app messages** - Mix of legitimate conversations and sophisticated scams
- 🤖 **AI moderation** - Context-aware safety analysis with structured results
- 🎯 **Structured results** - Violation types, safety flags, and human-readable explanations
- 🛡️ **Catch what keywords miss** - Romance scams, crypto pitches, and harassment detection

No keyword lists, no brittle rules, no false positives - just intelligent content understanding.

In [ ]:
import fenic as fc
from pydantic import BaseModel, Field
from typing import Literal

# ⚡ Configure for content moderation
session = fc.Session.get_or_create(fc.SessionConfig(
    app_name="content_moderation_demo",
    semantic=fc.SemanticConfig(
        language_models={
            "moderator": fc.OpenAILanguageModel(model_name="gpt-4o-mini", rpm=500, tpm=200_000),
            # "moderator": fc.GoogleDeveloperLanguageModel(model_name="gemini-2.5-flash-lite", rpm=1000, tpm=1_000_000),
            # "moderator": fc.AnthropicLanguageModel(model_name="claude-3-5-sonnet-20241022", rpm=500, tpm=200_000)
        }
    )
))

print("✅ Content moderation session configured")

## 📱 Step 1: Structured Moderation Schema

Define exactly what we want to extract from each message for safety analysis:

In [ ]:
# 🎯 Structured moderation schema using Pydantic
class ModerationResult(BaseModel):
    is_safe: bool = Field(description="Whether content is safe for the platform")
    violation_type: Literal["spam", "scam", "harassment", "safe"] = Field(description="Type of violation detected")
    reason: str = Field(description="Brief explanation")

print("✅ Moderation schema defined:")
print("   • is_safe: Boolean safety flag (True/False)")
print("   • violation_type: Category of violation (spam/scam/harassment/safe)")  
print("   • reason: Human-readable explanation")

## 💬 Step 2: Dating App Messages (Including Scams)

Mix of legitimate messages and sophisticated scams that keyword filters would miss:

In [ ]:
# 📱 Dating app messages with sophisticated scams
messages = session.create_dataframe([
    {"id": "M1", "content": "Hey beautiful! Want to chat privately? I can help you make easy money from home 💰"},
    {"id": "M2", "content": "This restaurant has amazing pasta! Perfect for a romantic dinner 🍝❤️"},
    {"id": "M3", "content": "I'm a successful entrepreneur. Own multiple properties. Let's grab coffee!"},
    {"id": "M4", "content": "You seem special. Here's my crypto investment strategy that made me rich..."},
    {"id": "M5", "content": "You're ugly and stupid. Nobody will ever love you. Why don't you just disappear already?"},
    {"id": "M6", "content": "CLICK HERE NOW! Amazing singles in your area! Meet them tonight! Limited time offer! ACT FAST!!!"}
])

print("📱 Dating App Messages:")
messages.show()
print("\nNotice: Mix of legitimate messages, scams, harassment, and spam!")

## 🛡️ Step 3: AI Content Moderation

Extract structured moderation results using semantic.extract with our schema:

In [ ]:
# 🛡️ AI moderation using structured extraction
moderated = messages.select(
    "id",
    fc.semantic.extract(
        "content",
        ModerationResult,
        model_alias="moderator"
    ).alias("moderation")
).cache()  # Cache moderation results

results = moderated.select(
    "id",
    moderated.moderation.is_safe.alias("safe"),
    moderated.moderation.violation_type.alias("violation"),
    moderated.moderation.reason.alias("reason")
)

print("🛡️ AI MODERATION RESULTS:")
results.show()

unsafe_count = results.filter(~fc.col("safe")).count()
print(f"\n🎯 CAUGHT {unsafe_count} unsafe messages!")
print("   💰 Money scam detected")
print("   🪙 Crypto pitch flagged") 
print("   😡 Harassment caught")
print("   📢 Spam identified")
print("   🍝 Restaurant recommendation approved")
print("\n💡 AI understands context and intent that keyword filters miss!")

In [ ]:
session.stop()